In [ ]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import importlib
importlib.reload(spot)
importlib.reload(roi_prop)
importlib.reload(intensity)

In [ ]:
### Specify directories to data files
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
output_dir = os.path.join(input_dir, "testfull-aug5")

# fixed image (directory - n5 format)
img_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 
# spot dir 
spot_dir = os.path.join(input_dir, "spots_pooled") # pool spots together; warpped and fixed
# bad roi
bad_roi = os.path.join(output_dir, "bad_roi_list.npy")
### end input

# output
out_roi = os.path.join(output_dir, "roi_all.csv") 
out_goodroi = os.path.join(output_dir, "roi.csv") 
out_spots = os.path.join(output_dir, "spots.csv")
out_intensity = os.path.join(output_dir, "intensity_c0_r2.csv")

In [ ]:
### Load segmentation mask
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

In [ ]:
### Extract ROI properties
print(lb.shape)
lb = lb[500:600,500:600,500:600]
# lb.shape # z, x,y 

In [ ]:
%%time
### Extract ROI properties
df = roi_prop.roi_prop(lb)

In [ ]:
df

In [ ]:
%%time
lb_stat_v2 = roi_prop.roi_prop_v2(lb)

In [ ]:
lb_stat_v2

In [ ]:
%%time
xyz = []
for item in lb_stat:
    xyz.append(item.centroid)
xyz = np.asarray(xyz)
# xyz

In [ ]:
lb_stat

In [ ]:
lb_id = np.unique(lb[lb != 0])

In [ ]:
lb_id

In [ ]:
df

In [ ]:
### Remove bad ROIs and save the file
df_filtered = df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi = df_filtered.roi.astype(int)
df_filtered.to_csv(out_goodroi)

In [ ]:
df.to_csv(out_roi)

In [ ]:
# ### Lipofuscin subtraction (optional)
# # Load spot files for lipofuscin subtraction
# channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
# channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# # Identify lipofuscin spots
# c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# # Remove lipofuscin spots
# true_pos_c0 = np.delete(channel_1, pAind, axis=0)
# true_pos_c1 = np.delete(channel_2, pBind, axis=0)

### spot extraction and save file
spotcount = spot.spot_counts(lb, spot_dir, verbose=False)
spotcount.to_csv(out_spots)

In [ ]:
### Intensity measurement and save file
df_intensity=intensity.measure_intensity(lb, img_dir, 'c0')
df_intensity.to_csv(out_intensity)

In [ ]:
test = pd.read_csv(out_spots, index_col=0)
for col in test:
    print(np.max(test[col]))
test